In [1]:
dataset_today =  'March 2024.xlsx'
dataset_pervious_day = 'Feb 2024.xlsx'

# Importing Data And Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
dataset = pd.read_excel(dataset_today)
df = dataset.copy()

In [4]:
dataset = pd.read_excel(dataset_pervious_day)
df_pervious = dataset.copy()

# Cleaning Part

### Extract the year and the month columns


In [5]:
df['day'] = pd.to_datetime(df['Date']).dt.day
d = list(df['day'].dropna().unique())

df['month'] = pd.to_datetime(df['Date']).dt.month_name()
m = list(df['month'].dropna().unique())

df['year'] = pd.to_datetime(df['Date']).dt.year
y = list(df['year'].dropna().unique())

# Change the directory in order to put all the excel files reports in the day folder

In [7]:
root_path = 'Desktop/'
os.mkdir(os.path.join(root_path, 'تحليل فئات' + m[-1]) )
os.chdir(os.path.join(root_path, 'تحليل فئات' + m[-1] ))

### Split product cat Columns

In [8]:
df[['Product origin', 'Cat1', 'cat2', 'cat3']]= df['Product Category'].str.split("/",expand=True)
df.drop(columns= {'Product Category'}, inplace = True)

df_pervious[['Product origin', 'Cat1', 'cat2', 'cat3']]= df_pervious['Product Category'].str.split("/",expand=True)
df_pervious.drop(columns= {'Product Category'}, inplace = True)



### Prepare the required Columns

In [9]:
df['discount amount'] = df['Quantity'] * df['Discount per Unit (abs)']
df['discount percentage'] = ((df['Discount (%)'] / 100) * df['Unit Price']) *df['Quantity']
df['Total discount'] = df['discount amount']  + df['discount percentage']
df['price before discount'] = df['Unit Price'] * df['Quantity']
df['Cost per unit'] = df['Cost'] * df['Quantity']
df['vat'] = df['Subtotal'] / 1.15
df['profit margin'] = df['vat'] - df['Cost'] 
df['Profit margin percentage'] = df['profit margin'] /df['vat']

In [10]:
df_pervious['discount amount'] = df_pervious['Quantity'] * df_pervious['Discount per Unit (abs)']
df_pervious['discount percentage'] = ((df_pervious['Discount (%)'] / 100) * df_pervious['Unit Price']) *df_pervious['Quantity']
df_pervious['Total discount'] = df_pervious['discount amount']  + df_pervious['discount percentage']
df_pervious['price before discount'] = df_pervious['Unit Price'] * df_pervious['Quantity']
df_pervious['Cost per unit'] = df_pervious['Cost'] * df_pervious['Quantity']
df_pervious['vat'] = df_pervious['Subtotal'] / 1.15
df_pervious['profit margin'] = df_pervious['vat'] - df_pervious['Cost'] 
df_pervious['Profit margin percentage'] = df_pervious['profit margin'] /df_pervious['vat']

### Cleaning the values of the branches to be in a better shape

In [11]:
df['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الدمام': 'راحتي الدمام'
                             ,'مبيعات نقاط بيع فرع بريدة الكبير 1' : 'فرع بريدة الكبير 1', 
                            'مبيعات نقاط بيع راحتي الرس': 'راحتي الرس',
                            'مبيعات نقاط  بيع راحتي  الرياض 1 اثاث': 'راحتي  الرياض 1 اثاث',
                             'مبيعات نقاط بيع راحتي حائل 1' : 'راحتي حائل 1',
                            'مبيعات نقاط بيع راحتي الطائف الكبير 1' : ' راحتي الطائف الكبير 1', 
                            'مبيعات نقاط بيع راحتي بريدة الكبير 2' : ' راحتي بريدة الكبير 2',
                            'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل', 
                            'مبيعات نقاط بيع راحتي حفر الباطن': 'راحتي حفر الباطن',
                            'مبيعات نقاط بيع راحتي حائل 2' : 'بيع راحتي حائل 2', 
                            'مبيعات نقاط بيع راحتي البدائع': 'راحتي البدائع',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الرياض2 مفروشات': 'راحتي الرياض2 مفروشات',
                            'مبيعات نقاط بيع راحتي الطائف الصغير 2' : 'راحتي الطائف الصغير 2',
                             'مبيعات نقاط بيع قصر البيان الرس': 'قصر البيان الرس',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني', 
                             'مبيعات فرع تبوك': 'فرع تبوك',
                            'مبيعات الموقع تبوك': 'الموقع تبوك'}, inplace = True)

In [12]:
df_pervious['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الدمام': 'راحتي الدمام'
                             ,'مبيعات نقاط بيع فرع بريدة الكبير 1' : 'فرع بريدة الكبير 1', 
                            'مبيعات نقاط بيع راحتي الرس': 'راحتي الرس',
                            'مبيعات نقاط  بيع راحتي  الرياض 1 اثاث': 'راحتي  الرياض 1 اثاث',
                             'مبيعات نقاط بيع راحتي حائل 1' : 'راحتي حائل 1',
                            'مبيعات نقاط بيع راحتي الطائف الكبير 1' : ' راحتي الطائف الكبير 1', 
                            'مبيعات نقاط بيع راحتي بريدة الكبير 2' : ' راحتي بريدة الكبير 2',
                            'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل', 
                            'مبيعات نقاط بيع راحتي حفر الباطن': 'راحتي حفر الباطن',
                            'مبيعات نقاط بيع راحتي حائل 2' : 'بيع راحتي حائل 2', 
                            'مبيعات نقاط بيع راحتي البدائع': 'راحتي البدائع',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الرياض2 مفروشات': 'راحتي الرياض2 مفروشات',
                            'مبيعات نقاط بيع راحتي الطائف الصغير 2' : 'راحتي الطائف الصغير 2',
                             'مبيعات نقاط بيع قصر البيان الرس': 'قصر البيان الرس',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني', 
                             'مبيعات فرع تبوك': 'فرع تبوك',
                            'مبيعات الموقع تبوك': 'الموقع تبوك'}, inplace = True)

### Prepare different pages 

#### Page of Deposits

In [13]:
df_deposit = df[df['Product origin'] == 'منتجات مصروفات الشركة']
df_deposit.to_excel('Service and other things report.xlsx', sheet_name='منتجات مصروفات الشركة',  index = False)
df.loc[df_deposit.index,"Quantity"] = 1

#### Page of services and installment

In [14]:
df_services = df[df['Product origin'] == 'خدمات-44 ']


In [15]:
df_installment = df[df['Product origin'] == 'خدمات التقسيط']
df_services =pd.concat([df_services, df_installment])

#### Get the additional discounts rows, put it in dataframe and save it as a file

In [16]:
df_all = df[df['Product origin'] == 'All']
df_addition_discount = df_all[df_all['Product'] == '[11223344] RS خصم اضافي للعميل']
df_all.drop(df_addition_discount.index, inplace = True)
df_services = pd.concat([df_services, df_all])
df_services.to_excel('service.xlsx')

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_5444\4004099315.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.drop(df_addition_discount.index, inplace = True)


In [17]:
df_all_pervious = df_pervious[df_pervious['Product origin'] == 'All']
df_addition_discount_pervious = df_pervious[df_pervious['Product'] == '[11223344] RS خصم اضافي للعميل']
df_all_pervious.drop(df_addition_discount_pervious.index, inplace = True)

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_5444\111177.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_pervious.drop(df_addition_discount_pervious.index, inplace = True)


#### Page of zero qunatity

In [18]:
df_0 = df[df['Quantity'] == 0 ]
df.drop(df_0.index, inplace = True)

In [19]:
df_0 = df_pervious[df_pervious['Quantity'] == 0 ]
df_pervious.drop(df_0.index, inplace = True)

#### Final Total

In [20]:
df_final_total = df[df['Product'] == 'الإجمالي النهائي' ]
df.drop(df_final_total.index, inplace = True)

In [21]:
df_final_total = df_pervious[df_pervious['Product'] == 'الإجمالي النهائي' ]
df_pervious.drop(df_final_total.index, inplace = True)

# Analytical Part

## Product Reports

### Drop additional discounts, services, and deposits from dataframe in order to get accurate results

In [22]:
df2 = df.copy()
df2.drop(df_addition_discount.index, inplace = True)
df2.drop(df_services.index, errors= 'ignore', inplace = True)
df2.drop(df_deposit.index, errors= 'ignore', inplace = True)
df2.dropna(inplace = True)

### Products By cat2 for the sales Manager

In [31]:
df['cat2'].unique()
data_sales = pd.DataFrame()
for x in list(df2['cat2'].unique()):
    category = df2[df2['cat2'] == x]
    df_Product_sold_units = pd.pivot_table(category,index=['Product'], values=('Quantity'), aggfunc = 'sum')
    df_Product_sold_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)
    df_Product_sold_units = df_Product_sold_units.merge(df[['Barcode', 'Product']].drop_duplicates(), how = 'inner', on = 'Product')
    df_Product_sold_units.rename(columns = {'Product': 'المنتج', 'Sold units': 'عدد الوحدات', 'Barcode': 'باركود'}, inplace = True)
    
    
    df_sales = pd.pivot_table(category,index=['Product'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات', 'Product': 'المنتج'}, inplace = True) 
    
    
    df_discounts = pd.pivot_table(category,index=['Product'], values=('Total discount'), aggfunc = 'mean')
    df_discounts.reset_index(inplace = True)
    df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات', 'Product': 'المنتج'}, inplace = True)
    
    df_all= df_Product_sold_units.merge(df_sales, on= 'المنتج', how = 'inner')
    df_all= df_all.merge(df_discounts, on = 'المنتج', how = 'inner')
    df_all.set_index('باركود', inplace = True)
    df_all= df_all[['المنتج'
                    , 'عدد الوحدات' 
                    ,'إجمالي الخصومات'
                    , 'إجمالي المبيعات']]
    
    df_all.sort_values(by = ['إجمالي المبيعات', 'عدد الوحدات'], ascending = False ,inplace = True)
    df_all.loc['الاجمالي', 'المنتج'] =  df_sales['المنتج'].count()
    df_all.loc['الاجمالي', 'عدد الوحدات'] =  df_all['عدد الوحدات'].sum()
    df_all.loc['الاجمالي', 'إجمالي الخصومات'] = df_all['إجمالي الخصومات'].sum() 
    df_all.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()    
    if os.path.exists('منتجات بالفئة 2 المدير المبيعات' + '.xlsx') == False:
        df_all.to_excel('منتجات بالفئة 2 المدير المبيعات' + '.xlsx', sheet_name= x)
    else:
        with pd.ExcelWriter(('منتجات بالفئة 2 المدير المبيعات' + '.xlsx'), engine='openpyxl', mode='a') as writer:
            df_all.to_excel(writer, sheet_name= x)  

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

### Products By cat2 for the FO

In [33]:
df['cat2'].unique()
data_sales = pd.DataFrame()
for x in list(df2['cat2'].unique()):
    category = df2[df2['cat2'] == x]
    df_Product_sold_units = pd.pivot_table(category,index=['Product'], values=('Quantity'), aggfunc = 'sum')
    df_Product_sold_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)
    df_Product_sold_units = df_Product_sold_units.merge(df[['Barcode', 'Product']].drop_duplicates(), how = 'inner', on = 'Product')
    df_Product_sold_units.rename(columns = {'Product': 'المنتج', 'Sold units': 'عدد الوحدات', 'Barcode': 'باركود'}, inplace = True)
    
#    df_cost = pd.pivot_table(category,index=['Product'], values=('Cost'), aggfunc = 'sum')
#    df_cost.reset_index(inplace = True)
#    df_cost.rename(columns ={'Cost':'إجمالي التكلفة', 'Product': 'المنتج'}, inplace = True)
    
    df_sales = pd.pivot_table(category,index=['Product'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات', 'Product': 'المنتج'}, inplace = True) 

    df_profit = pd.pivot_table(category,index=['Product'], values=('Profit margin percentage'), aggfunc = 'mean')
    df_profit.reset_index(inplace = True)
    df_profit.rename(columns ={'Profit margin percentage':'متوسط هامش الربح', 'Product': 'المنتج'}, inplace = True)
    
    
    df_discounts = pd.pivot_table(category,index=['Product'], values=('Total discount'), aggfunc = 'mean')
    df_discounts.reset_index(inplace = True)
    df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات', 'Product': 'المنتج'}, inplace = True)
    
    df_all= df_Product_sold_units.merge(df_sales, on= 'المنتج', how = 'inner')
    #df_all= df_all.merge(df_cost, on = 'المنتج', how = 'inner')
    df_all= df_all.merge(df_profit, on = 'المنتج', how = 'inner')
    df_all= df_all.merge(df_discounts, on = 'المنتج', how = 'inner')
    df_all.set_index('باركود', inplace = True)
    df_all= df_all[['المنتج'
                    , 'عدد الوحدات' 
                    , 'إجمالي المبيعات'
                    ,'إجمالي الخصومات'
                    , 'متوسط هامش الربح']]

    df_all.sort_values(by = ['إجمالي المبيعات', 'عدد الوحدات'], ascending = False ,inplace = True)
    df_all.loc['الاجمالي', 'المنتج'] =  df_sales['المنتج'].count()
    df_all.loc['الاجمالي', 'عدد الوحدات'] =  df_all['عدد الوحدات'].sum()
    df_all.loc['الاجمالي', 'إجمالي الخصومات'] = df_all['إجمالي الخصومات'].sum() 
    df_all.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()  
    df_all.loc['الاجمالي', 'متوسط هامش الربح'] = df_all['متوسط هامش الربح'].mean()  
    
    if os.path.exists('منتجات بالفئة 2 المدير المالي' + '.xlsx') == False:
        df_all.to_excel('منتجات بالفئة 2 المدير المالي' + '.xlsx', sheet_name= x)
    else:
        with pd.ExcelWriter(('منتجات بالفئة 2 المدير المالي' + '.xlsx'), engine='openpyxl', mode='a') as writer:
            df_all.to_excel(writer, sheet_name= x)  


C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

### Products By cat1 for the FO

In [34]:
data_sales = pd.DataFrame()
for x in list(df2['Cat1'].unique()):
    category = df2[df2['Cat1'] == x]
    df_Product_sold_units = pd.pivot_table(category,index=['Product'], values=('Quantity'), aggfunc = 'sum')
    df_Product_sold_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)
    df_Product_sold_units = df_Product_sold_units.merge(df[['Barcode', 'Product']].drop_duplicates(), how = 'inner', on = 'Product')
    df_Product_sold_units.rename(columns = {'Product': 'المنتج', 'Sold units': 'عدد الوحدات', 'Barcode': 'باركود'}, inplace = True)
    
#    df_cost = pd.pivot_table(category,index=['Product'], values=('Cost'), aggfunc = 'sum')
#    df_cost.reset_index(inplace = True)
#    df_cost.rename(columns ={'Cost':'إجمالي التكلفة', 'Product': 'المنتج'}, inplace = True)
    
    df_sales = pd.pivot_table(category,index=['Product'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات', 'Product': 'المنتج'}, inplace = True) 

    df_profit = pd.pivot_table(category,index=['Product'], values=('Profit margin percentage'), aggfunc = 'mean')
    df_profit.reset_index(inplace = True)
    df_profit.rename(columns ={'Profit margin percentage':'متوسط هامش الربح', 'Product': 'المنتج'}, inplace = True)
    
    
    df_discounts = pd.pivot_table(category,index=['Product'], values=('Total discount'), aggfunc = 'mean')
    df_discounts.reset_index(inplace = True)
    df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات', 'Product': 'المنتج'}, inplace = True)
    
    df_all= df_Product_sold_units.merge(df_sales, on = 'المنتج', how = 'inner')
#    df_all= df_all.merge(df_cost, on = 'المنتج', how = 'inner')
    df_all= df_all.merge(df_profit, on = 'المنتج', how = 'inner')
    df_all= df_all.merge(df_discounts, on = 'المنتج', how = 'inner')
    df_all.set_index('باركود', inplace = True)
    df_all= df_all[['المنتج'
                    , 'عدد الوحدات' 
                    , 'إجمالي المبيعات'
                    ,'إجمالي الخصومات'
                    , 'متوسط هامش الربح']]

    df_all.sort_values(by = ['إجمالي المبيعات', 'عدد الوحدات'], ascending = False ,inplace = True)
    df_all.loc['الاجمالي', 'المنتج'] =  df_sales['المنتج'].count()
    df_all.loc['الاجمالي', 'عدد الوحدات'] =  df_all['عدد الوحدات'].sum()
    df_all.loc['الاجمالي', 'إجمالي الخصومات'] = df_all['إجمالي الخصومات'].sum() 
    df_all.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()  
    df_all.loc['الاجمالي', 'متوسط هامش الربح'] = df_all['متوسط هامش الربح'].mean()  
    
        
    if os.path.exists('منتجات بالفئة 1' + '.xlsx') == False:
        df_all.to_excel('منتجات بالفئة 1' + '.xlsx', sheet_name= x)
    else:
        with pd.ExcelWriter(('منتجات بالفئة 1'  + '.xlsx'), engine='openpyxl', mode='a') as writer:
            df_all.to_excel(writer, sheet_name= x)  

### Category Analysis

In [35]:
df_Product_sold_units = pd.pivot_table(df2,index=['cat2'], values=('Quantity'), aggfunc = 'sum')
df_Product_sold_units.reset_index(inplace = True)
df_Product_sold_units.rename(columns ={'Quantity':'عدد الوحدات', 
                                          'cat2': 'الفئة'}, inplace = True)
    
df_sales = pd.pivot_table(df2,index=['cat2'], values=('Subtotal'), aggfunc = 'sum')
df_sales.reset_index(inplace = True)
df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات', 'cat2': 'الفئة'}, inplace = True) 

df_profit = pd.pivot_table(df2,index=['cat2'], values=('Profit margin percentage'), aggfunc = 'mean')
df_profit.reset_index(inplace = True)
df_profit.rename(columns ={'Profit margin percentage':'متوسط هامش الربح', 'cat2': 'الفئة'}, inplace = True)

df_discounts = pd.pivot_table(df2,index=['cat2'], values=('Total discount'), aggfunc = 'mean')
df_discounts.reset_index(inplace = True)
df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات', 'cat2': 'الفئة'}, inplace = True)

df_all= df_Product_sold_units.merge(df_sales, on= 'الفئة', how = 'inner')
df_all= df_all.merge(df_profit, on = 'الفئة', how = 'inner')
df_all= df_all.merge(df_discounts, on = 'الفئة', how = 'inner')
df_all.reset_index(inplace = True)
df_all= df_all.sort_values(by = ['إجمالي المبيعات'], ascending = False)
df_all= df_all[['الفئة'
                , 'عدد الوحدات' 
                , 'إجمالي المبيعات'
                ,'إجمالي الخصومات'
                , 'متوسط هامش الربح']]

        
if os.path.exists('تحليل الفئات' + '.xlsx') == False:
    df_all.to_excel('تحليل الفئات' + '.xlsx', sheet_name= x)
else:
    with pd.ExcelWriter(('تحليل الفئات'  + '.xlsx'), engine='openpyxl', mode='a') as writer:
        df_all.to_excel(writer)  

### Category Analysis Per Branch

In [40]:
data_sales = pd.DataFrame()
for x in list(df2['فريق المبيعات'].unique()):
    df_Product_sold_units = pd.pivot_table(df2,index=['cat2'], values=('Quantity'), aggfunc = 'sum')
    df_Product_sold_units.reset_index(inplace = True)
    df_Product_sold_units.rename(columns ={'Quantity':'عدد الوحدات', 
                                           'cat2': 'الفئة'}, inplace = True)
    
    df_sales = pd.pivot_table(df2,index=['cat2'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات', 'cat2': 'الفئة'}, inplace = True) 

    df_profit = pd.pivot_table(df2,index=['cat2'], values=('Profit margin percentage'), aggfunc = 'mean')
    df_profit.reset_index(inplace = True)
    df_profit.rename(columns ={'Profit margin percentage':'متوسط هامش الربح', 'cat2': 'الفئة'}, inplace = True)

    df_discounts = pd.pivot_table(df2,index=['cat2'], values=('Total discount'), aggfunc = 'mean')
    df_discounts.reset_index(inplace = True)
    df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات', 'cat2': 'الفئة'}, inplace = True)

    df_all= df_Product_sold_units.merge(df_sales, on= 'الفئة', how = 'inner')
    df_all= df_all.merge(df_profit, on = 'الفئة', how = 'inner')
    df_all= df_all.merge(df_discounts, on = 'الفئة', how = 'inner')
    df_all.reset_index(inplace = True)
    df_all= df_all.sort_values(by = ['إجمالي المبيعات'], ascending = False)
    df_all= df_all[['الفئة'
                , 'عدد الوحدات' 
                , 'إجمالي المبيعات'
                ,'إجمالي الخصومات'
                , 'متوسط هامش الربح']]
    
    if os.path.exists(('تحليل الفئات' + ' في كل فرع ' +'.xlsx')) == False:
        df_all.to_excel(('تحليل الفئات' + ' في كل فرع ' +'.xlsx'), sheet_name= x)
    else:
        with pd.ExcelWriter(('تحليل الفئات' + ' في كل فرع ' +'.xlsx'), engine='openpyxl', mode='a') as writer:
            df_all.to_excel(writer, sheet_name= x)  

### Category Analysis Per category

In [41]:
data_sales = pd.DataFrame()
for x in list(df2['cat2'].unique()):
    category = df2[df2['cat2'] == x]
    df_Product_sold_units = pd.pivot_table(category,index=['فريق المبيعات'], values=('Quantity'), aggfunc = 'sum')
    df_Product_sold_units.reset_index(inplace = True)
    df_Product_sold_units.rename(columns ={'Quantity':'عدد الوحدات'}, inplace = True)
    
    df_sales = pd.pivot_table(category,index=['فريق المبيعات'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات'}, inplace = True) 

    #df_profit = pd.pivot_table(df2,index=['فريق المبيعات'], values=('Profit margin percentage'), aggfunc = 'mean')
    #df_profit.reset_index(inplace = True)
    #df_profit.rename(columns ={'Profit margin percentage':'متوسط هامش الربح'}, inplace = True)

    #df_discounts = pd.pivot_table(df2,index=['فريق المبيعات'], values=('Total discount'), aggfunc = 'mean')
    #df_discounts.reset_index(inplace = True)
    #df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات'}, inplace = True)

    df_all= df_Product_sold_units.merge(df_sales, on= 'فريق المبيعات', how = 'inner')
    #df_all= df_all.merge(df_profit, on = 'فريق المبيعات', how = 'inner')
    #df_all= df_all.merge(df_discounts, on = 'فريق المبيعات', how = 'inner')
    df_all.set_index('فريق المبيعات', inplace = True)
    df_all.sort_values(by = ['عدد الوحدات', 'إجمالي المبيعات'], ascending = False, inplace = True)   
    df_all.reset_index(inplace = True)
    df_all.reset_index(inplace = True)
    
    df_all.rename(columns = {'index': 'الترتيب' }, inplace = True)
    df_all.set_index('الترتيب', inplace = True)
    df_all= df_all[['فريق المبيعات'
                , 'عدد الوحدات' 
                , 'إجمالي المبيعات'
            #    ,'إجمالي الخصومات'
             #   , 'متوسط هامش الربح'
                   ]]
    
    if os.path.exists((' تحليل الفئات للفروع في كل فئة'  +'.xlsx')) == False:
        df_all.to_excel((' تحليل الفئات للفروع في كل فئة' +'.xlsx'), sheet_name= x)
    else:
        with pd.ExcelWriter((' تحليل الفئات للفروع في كل فئة' +'.xlsx'), engine='openpyxl', mode='a') as writer:
            df_all.to_excel(writer, sheet_name= x)  

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

# Employees Reports

#### Perpare the dataFrame we will work with 

In [42]:
df_addition_discount.loc[ : ,'Quantity'] = 0
df3 =df.copy()
df3.loc[list(df_addition_discount.index) ,'Quantity'] = 0
df3.drop(df_deposit.index, inplace =True, errors = 'ignore') 
df3.drop(df_services.index, inplace =True, errors = 'ignore') 


#### Remove the Websites sales

In [43]:
xx = df3[df3['فريق المبيعات'] == 'متجر راحتي الالكتروني']
df3.drop(xx.index, axis = 0, inplace = True)
xx = df3[df3['فريق المبيعات'] == 'المتجر الإلكتروني']
df3.drop(xx.index, axis = 0, inplace = True)

In [44]:
xx = df3[df3['فريق المبيعات'] == 'قصر البيان الإلكتروني']
df3.drop(xx.index, axis = 0, inplace = True)
xx = df3[df3['فريق المبيعات'] == 'الموقع تبوك']
df3.drop(xx.index, axis = 0, inplace = True)

#### Remove Branch Whatsup accounts from employees

In [45]:
df3.drop(df3[df3['Order Ref/Cashier'].isna()].index, inplace = True)
whatsup = df3[df3['Order Ref/Cashier'].str.contains('واتس')]
df3.drop(whatsup.index, inplace = True)

##### Employees Sold unit

In [46]:
Employee_sold_units = pd.pivot_table(df3,index=['Order Ref/Cashier'], values=(['Quantity']), aggfunc = 'sum')
Employee_sold_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)
df_Employee_sold_units= Employee_sold_units.merge(df3[['فريق المبيعات',
                                                'Order Ref/Cashier']].drop_duplicates(), on ='Order Ref/Cashier', how= 'inner')

##### solve the problem of the movement of salesmen from one branch to another at the same week

In [47]:
zzz = pd.DataFrame(df_Employee_sold_units.groupby('Order Ref/Cashier')['فريق المبيعات'].apply(lambda x: list(x)))
zzz.reset_index(inplace = True)
zzz = zzz.merge(Employee_sold_units, on ='Order Ref/Cashier', how= 'inner')
zzz = zzz[['Order Ref/Cashier', 'Sold units', 'فريق المبيعات']]

In [48]:
list_employee_cor = []
employee_dup_index = []
employee_dup_indexes = list(df_Employee_sold_units[df_Employee_sold_units['Order Ref/Cashier'].duplicated()].index)
employee_dup_names = df_Employee_sold_units.loc[employee_dup_indexes, 'Order Ref/Cashier']


for e in employee_dup_names:
    list_employee_cor.append(zzz[zzz['Order Ref/Cashier'] == e].index[0])

for u in range(0, len(employee_dup_indexes)):
    employee_dup_index.append(employee_dup_indexes[u])    

for i in range(0 , len(list_employee_cor)):
    o = list_employee_cor[i]
    p = employee_dup_index[i]
    df_Employee_sold_units.loc[p, 'فريق المبيعات'] = str(zzz.loc[o, 'فريق المبيعات'])

df_Employee_sold_units.drop_duplicates(subset = ['Order Ref/Cashier'], keep = 'last', inplace = True)

In [49]:
df_Employee_sold_units.drop(columns = 'Sold units', inplace = True)
df_Employee_sold_units.rename(columns = {'Order Ref/Cashier': 'الموظف'}, inplace = True)

## Employees Sales Report


In [50]:
for x in list(df3['Cat1'].dropna().unique()):
    category = df3[df3['Cat1'] == x]
    df_units = pd.pivot_table(category,index=['Order Ref/Cashier'], values=('Quantity'), aggfunc = 'sum')
    df_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)
    df_units.reset_index(inplace = True)
    df_units.rename(columns = {'Order Ref/Cashier': 'الموظف', 'Sold units': 'عدد الوحدات', 'Barcode': 'باركود'}, inplace = True)
    
    df_cost = pd.pivot_table(category,index=['Order Ref/Cashier'], values=('Cost'), aggfunc = 'sum')
    df_cost.reset_index(inplace = True)
    df_cost.rename(columns ={'Cost':'إجمالي التكلفة', 'Order Ref/Cashier': 'الموظف'}, inplace = True)
    
    df_sales = pd.pivot_table(category,index=['Order Ref/Cashier'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات', 'Order Ref/Cashier': 'الموظف'}, inplace = True) 

    df_profit = pd.pivot_table(category,index=['Order Ref/Cashier'], values=('Profit margin percentage'), aggfunc = 'mean')
    df_profit.reset_index(inplace = True)
    df_profit.rename(columns ={'Profit margin percentage':'متوسط هامش الربح', 'Order Ref/Cashier': 'الموظف'}, inplace = True)
    
    
    df_discounts = pd.pivot_table(category,index=['Order Ref/Cashier'], values=('Total discount'), aggfunc = 'mean')
    df_discounts.reset_index(inplace = True)
    df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات', 'Order Ref/Cashier': 'الموظف'}, inplace = True)
    
    df_all= df_units.merge(df_cost, on =['الموظف'], how = 'inner')
    df_all= df_all.merge(df_sales, on = ['الموظف'], how = 'inner')
    df_all= df_all.merge(df_profit, on = ['الموظف'], how = 'inner')
    df_all= df_all.merge(df_discounts, on = ['الموظف'], how = 'inner')
    
    df_all= df_all.merge(df_Employee_sold_units, on = 'الموظف', how = 'inner')
    df_all.set_index('الموظف', inplace = True)
    df_all= df_all[['فريق المبيعات'
                    , 'عدد الوحدات' 
                    , 'إجمالي التكلفة'
                    , 'إجمالي المبيعات'
                    ,'إجمالي الخصومات'
                    , 'متوسط هامش الربح']]
    
    if os.path.exists(('1 مجمع موظفين بالفئة'  +'.xlsx')) == False:
        df_all.to_excel('1 مجمع موظفين بالفئة' +'.xlsx', sheet_name= x)  
    else:    
        with pd.ExcelWriter(('1 مجمع موظفين بالفئة'  +'.xlsx'), engine='openpyxl', mode='a') as writer:
            df_all.to_excel(writer, sheet_name= x)  

In [51]:
for x in list(df3['cat2'].dropna().unique()):
    category = df3[df3['cat2'] == x]
    df_units = pd.pivot_table(category,index=['Order Ref/Cashier', 'cat2'], values=('Quantity'), aggfunc = 'sum')
    df_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)
    df_units.reset_index(inplace = True)
    df_units.rename(columns = {'Order Ref/Cashier': 'الموظف', 'Sold units': 'عدد الوحدات', 'Barcode': 'باركود'}, inplace = True)
    
    df_cost = pd.pivot_table(category,index=['Order Ref/Cashier', 'cat2'], values=('Cost'), aggfunc = 'sum')
    df_cost.reset_index(inplace = True)
    df_cost.rename(columns ={'Cost':'إجمالي التكلفة', 'Order Ref/Cashier': 'الموظف'}, inplace = True)
    
    df_sales = pd.pivot_table(category,index=['Order Ref/Cashier', 'cat2'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات', 'Order Ref/Cashier': 'الموظف'}, inplace = True) 

    df_profit = pd.pivot_table(category,index=['Order Ref/Cashier', 'cat2'], values=('Profit margin percentage'), aggfunc = 'mean')
    df_profit.reset_index(inplace = True)
    df_profit.rename(columns ={'Profit margin percentage':'متوسط هامش الربح', 'Order Ref/Cashier': 'الموظف'}, inplace = True)
    
    
    df_discounts = pd.pivot_table(category,index=['Order Ref/Cashier', 'cat2'], values=('Total discount'), aggfunc = 'mean')
    df_discounts.reset_index(inplace = True)
    df_discounts.rename(columns ={'Total discount':'إجمالي الخصومات', 'Order Ref/Cashier': 'الموظف'}, inplace = True)
    
    df_all= df_units.merge(df_cost, on =['الموظف', 'cat2'], how = 'inner')
    df_all= df_all.merge(df_sales, on = ['الموظف', 'cat2'], how = 'inner')
    df_all= df_all.merge(df_profit, on = ['الموظف', 'cat2'], how = 'inner')
    df_all= df_all.merge(df_discounts, on = ['الموظف', 'cat2'], how = 'inner')
    df_all.rename(columns = {'cat2': 'الفئة'}, inplace = True)
    
    df_all= df_all.merge(df_Employee_sold_units, on = 'الموظف', how = 'inner')
    df_all.set_index('الموظف', inplace = True)
    df_all= df_all[['فريق المبيعات'
                    ,'الفئة'
                    , 'عدد الوحدات' 
                    , 'إجمالي التكلفة'
                    , 'إجمالي المبيعات'
                    ,'إجمالي الخصومات'
                    , 'متوسط هامش الربح']]
    
    if os.path.exists(('2 مجمع موظفين بالفئة'  +'.xlsx')) == False:
        df_all.to_excel('2 مجمع موظفين بالفئة'  +'.xlsx', sheet_name= x)  
    else:    
        with pd.ExcelWriter(('2 مجمع موظفين بالفئة' +'.xlsx'), engine='openpyxl', mode='a') as writer:
            df_all.to_excel(writer, sheet_name= x)  

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may n